# Load Dependency

In [13]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [14]:
np.__version__

'1.19.2'

In [15]:
cv2.__version__

'4.5.4-dev'

# Landmark Display

In [2]:
mp_hands = mp.solutions.hands # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
def draw_landmark( image ):
    # Make detections
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = hands.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
                
    return image , results

In [ ]:
cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_hands.Hands( max_num_hands = 1,
                    min_detection_confidence=0.5, 
                    min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():

        # Read feed
        success, image = cap.read()
        
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        image ,results = draw_landmark( image )
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

# Extract Key Features

In [ ]:
landmark_feature = []

for hand_landmarks in results.multi_hand_landmarks:
    for landmark in hand_landmarks.landmark:
        test = np.array([landmark.x, landmark.y, landmark.z])
        landmark_feature.append(test)

In [ ]:
np.array(landmark_feature).shape

# Train Data Folder

In [4]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Train_Data') 

# Actions that we try to detect
actions = np.array(['jamjam' , 'hi' , 'circle'])
#actions = np.array(['jamjam', 'Hi'])

# Thirty videos worth of data
no_sequences = 10

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [ ]:
for action in actions: 
    dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    
    #for sequence in range(1,no_sequences+1):
    for sequence in range(0,no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

In [8]:
cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_hands.Hands( max_num_hands = 1,
                    min_detection_confidence=0.5, 
                    min_tracking_confidence=0.5) as hands:
    
    if cap.isOpened():
        
        for action in actions[1:]:
            
            # Loop through sequences aka videos
            for sequence in range(start_folder, start_folder+no_sequences):
                
                # Loop through video length aka sequence length
                for frame_num in range(sequence_length):
        
                    # Read feed
                    success, image = cap.read()

                    image ,results = draw_landmark( image )  

                    cv2.imshow('OpenCV Feed', image)
                    
                    # NEW Apply wait logic
                    if frame_num == 0: 
                        cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                        cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                        cv2.imshow('OpenCV Feed', image)
                        #cv2.waitKey(500)
                        cv2.waitKey(0)
                        
                    else: 
                        cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                        cv2.imshow('OpenCV Feed', image)

                    # NEW Export keypoints
                    landmark_feature = []

                    for hand_landmarks in results.multi_hand_landmarks:
                        for landmark in hand_landmarks.landmark:
                            test = np.array([landmark.x, landmark.y, landmark.z])
                            landmark_feature.append(test)
                    
                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                    print( np.array(landmark_feature).flatten().shape )
                    np.save(npy_path, np.array(landmark_feature).flatten())

                    # Break gracefully
                    if cv2.waitKey(10) & 0xFF == ord('q'):
                        break

    cap.release()
    cv2.destroyAllWindows()

TypeError: 'NoneType' object is not iterable

In [9]:
cap.release()
cv2.destroyAllWindows()